In [ ]:
!pip install --upgrade keras
!pip install --upgrade keras tensorflow
import os
import math
import numpy as np
import librosa
import json
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install tensorboard
!pip install visualkeras
import visualkeras
from keras.utils import plot_model
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Audio
from scipy.io import wavfile
import scipy
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import random

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Mel_Spectrograms/Train'
test_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Mel_Spectrograms/Test'

num_classes = 8
height = 128
n_fft = 2048
hop_length = 512
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator()

batch_size = 32

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

model = tf.keras.Sequential([
    tf.keras.layers.Reshape((width, height * channels), input_shape=(height, width, channels)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu', return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu')),  # Existing LSTM layer
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 200
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    train_generator.reset()

    model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=1,
        verbose=1)

loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

model.save('al-emadi_rnn_mcc_standardized.keras')

Found 2868 images belonging to 8 classes.
Found 714 images belonging to 8 classes.
Epoch 1/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 493s 5s/step - accuracy: 0.2190 - loss: 2.1707
Epoch 2/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 58s 593ms/step - accuracy: 0.4347 - loss: 1.6943
Epoch 3/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 58s 598ms/step - accuracy: 0.4934 - loss: 1.5393
Epoch 4/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 59s 619ms/step - accuracy: 0.5210 - loss: 1.4035
Epoch 5/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 58s 602ms/step - accuracy: 0.5498 - loss: 1.3354
Epoch 6/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 57s 587ms/step - accuracy: 0.5967 - loss: 1.1437
Epoch 7/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 57s 586ms/step - accuracy: 0.6055 - loss: 1.1136
Epoch 8/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 56s 577ms/step - accuracy: 0.6127 - loss: 1.0889
Epoch 9/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 57s 583ms/step - accuracy: 0.6082 - loss: 1.1208
Epoch 10/200
90/90 ━━━━━━━━━━━━━━━━━━━━ 57s 580ms/step - accuracy: 0.6360 - loss: 1.0362
Epoch 11/200
90/90 ━━━━━━━━━━━━━━━━━━━

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

num_classes = 8
height = 128
n_fft = 2048
hop_length = 512
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

batch_size = 32

model = tf.keras.models.load_model('al-emadi_rnn_mcc_standardized.keras')

test_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Mel_Spectrograms/Test'

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels
true_classes = test_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Calculate accuracy for each class
class_accuracy = {}
for i, class_label in enumerate(class_labels):
    correct_indices = np.where(true_classes == i)[0]
    correct_predictions = np.sum(predicted_classes[correct_indices] == i)
    total_samples = len(correct_indices)
    class_accuracy[class_label] = correct_predictions / total_samples if total_samples > 0 else 0.0

# Print accuracy for each class
print("\nAccuracy for each class:")
for class_label in class_labels:
    print(f"{class_label}: {class_accuracy[class_label]:.4f}")

# Print classification report (includes precision, recall, F1-score)
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

Found 714 images belonging to 8 classes.
23/23 ━━━━━━━━━━━━━━━━━━━━ 16s 597ms/step - accuracy: 0.8840 - loss: 1.0177
Test Loss: 0.5097439885139465
Test Accuracy: 0.9369747638702393
23/23 ━━━━━━━━━━━━━━━━━━━━ 14s 550ms/step

Confusion Matrix:
[[ 39   2   0   0   1   9   1   0]
 [  0  52   0   0   0   0   0   0]
 [  0   3  31   0   0  17   1   0]
 [  0   0   0  46   3   0   1   0]
 [  0   0   0   0  47   2   1   0]
 [  0   1   0   1   1 353   1   0]
 [  0   0   0   0   0   0  51   0]
 [  0   0   0   0   0   0   0  50]]

Accuracy for each class:
Bebop: 0.7500
Crazyflie: 1.0000
Mambo: 0.5962
Matrice100: 0.9200
Mavic: 0.9400
NonDrone: 0.9888
Phantom: 1.0000
Spark: 1.0000

Classification Report:
              precision    recall  f1-score   support

       Bebop       1.00      0.75      0.86        52
   Crazyflie       0.90      1.00      0.95        52
       Mambo       1.00      0.60      0.75        52
  Matrice100       0.98      0.92      0.95        50
       Mavic       0.90      0

In [ ]:
import os

# Define the root directory of your mounted Google Drive
drive_root = '/content/drive/MyDrive/'

def find_directories_with_double_wav(root_directory):

    file_counter = 0
    for root, dirs, files in os.walk(root_directory):
        for file in files:
            if file.endswith('.wav.wav'):
                file_counter+=1
                directory_path = os.path.abspath(root)
                print(f"Found file: {file} in directory: {directory_path}")

    print(file_counter)

# Call the function to search from the root of your Google Drive
find_directories_with_double_wav(drive_root)

Found file: 1-137-A-320.wav 1-137-A-321.wav.wav in directory: /content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_2000_2sec/Train/Category 0
Found file: 1-137-A-322.wav 1-137-A-323.wav.wav in directory: /content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_2000_2sec/Train/Category 0
Found file: 1-137-A-324.wav 1-977-A-390.wav.wav in directory: /content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_2000_2sec/Train/Category 0
Found file: 1-977-A-391.wav 1-977-A-392.wav.wav in directory: /content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_2000_2sec/Train/Category 0
Found file: 1-977-A-393.wav 1-977-A-394.wav.wav in directory: /content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_2000_2sec/Train/Category 0
Found file: 1-1791-A-260.wav 1-1791-A-261.wav.wav in directory: /content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_2000_2sec/Train/Category 0
Found file: 1-1791-A-262.wav 1-1791-A-263.wav.wav in dir

In [ ]:
import os

# Define the directory path where you want to count the files
directory_path = '/content/drive/MyDrive/YourDirectory/'

def count_wav_wav_files(directory):
    wav_wav_count = 0
    for file in os.listdir(directory):
        if file.endswith('.wav.wav'):
            wav_wav_count += 1
    return wav_wav_count

# Call the function to count .wav.wav files in the specified directory
count = count_wav_wav_files('/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_2000_2sec/Test/Category 0')
print(f"Number of .wav.wav files in '{directory_path}': {count}")

Number of .wav.wav files in '/content/drive/MyDrive/YourDirectory/': 233
